In [1]:
#q1
! hdfs dfsadmin -fs hdfs://boss:9000 -report | grep 'Live datanodes'

Live datanodes (2):


In [2]:
! rm -rf "/nb/hdma-wi-2021.csv"
! wget "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv" "/nb/hdma-wi-2021.csv"

import pyarrow.csv as pv
import pyarrow.parquet as pq

csv_path = "/nb/hdma-wi-2021.csv"
parquet_path = "/nb/hdma-wi-2021.parquet"

table = pv.read_csv(csv_path)
pq.write_table(table, parquet_path)

! hdfs dfs -rm -f hdfs://boss:9000/single.parquet
! hdfs dfs -rm -f hdfs://boss:9000/double.parquet

! hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -copyFromLocal {parquet_path} hdfs://boss:9000/single.parquet
! hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -copyFromLocal {parquet_path} hdfs://boss:9000/double.parquet

--2024-10-19 00:34:32--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv’

hdma-wi-2021.csv    100%[===================>] 166.84M   157MB/s    in 1.1s    

2024-10-19 00:34:34 (157 MB/s) - ‘hdma-wi-2021.csv’ saved [174944099/174944099]

/nb/hdma-wi-2021.csv: Scheme missing.
FINISHED --2024-10-19 00:34:34--
Total wall clock time: 1.2s
Downloaded: 1 files, 167M in 1.1s (157 MB/s)
2024-10-19 00:34:49,063 INFO hdfs.DFSOutputStream: Exception while adding a block
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.NotReplicatedYetException): Not replicated yet: /single.parquet._COPYING_
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.validateAddBlock(FSDirWriteFileOp.java:181)
	at org.ap

In [3]:
#q2
! hdfs dfs -du -h hdfs://boss:9000/single.parquet
! hdfs dfs -du -h hdfs://boss:9000/double.parquet

15.9 M  15.9 M  hdfs://boss:9000/single.parquet
15.9 M  31.7 M  hdfs://boss:9000/double.parquet


In [4]:
#q3
import requests
hdfs_url = "http://boss:9870/webhdfs/v1/single.parquet"
params = {
    "op": "GETFILESTATUS"
}
response = requests.get(hdfs_url, params=params)
file_status = response.json()
file_status

{'FileStatus': {'accessTime': 1729298088417,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 16642976,
  'modificationTime': 1729298090207,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [5]:
#q4
hdfs_url = "http://boss:9870/webhdfs/v1/single.parquet"
params = {
    "op": "OPEN",
    "offset": 0,
    "noredirect": "true"
}
response = requests.get(hdfs_url, params=params)
location = response.json()['Location']
location

'http://9322cd244a1a:9864/webhdfs/v1/single.parquet?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [6]:
#q5
import re
from collections import defaultdict

def get_block_locations(file_path, total_length, block_size):
    webhdfs_url = f"http://boss:9870/webhdfs/v1{file_path}"
    block_distribution = defaultdict(int)

    for offset in range(0, total_length, block_size):
        params = {
            "op": "OPEN",
            "offset": offset,
            "noredirect": "true"
        }
        
        response = requests.get(webhdfs_url, params=params)  
        if response.status_code == 200:
            response = response.json()
            location = response['Location']
            container_id = re.search(r'http://([^:]+):', location).group(1)
            block_distribution[container_id] += 1
    return dict(block_distribution)

status_params = {"op": "GETFILESTATUS"}
response = requests.get("http://boss:9870/webhdfs/v1/single.parquet", params=status_params)
file_status = response.json()['FileStatus']
total_length = file_status['length']
block_size = file_status['blockSize']
block_distribution = get_block_locations(file_path="/single.parquet", total_length=total_length, block_size=block_size)
block_distribution

{'9322cd244a1a': 6, '2a68af0b41a3': 10}

In [7]:
#q6
def get_block_locations(file_path, length):
    webhdfs_url = f"http://boss:9870/webhdfs/v1{file_path}"
    params = {
        "op": "GETFILEBLOCKLOCATIONS",
        "length": length
    }
    
    response = requests.get(webhdfs_url, params=params)   
    if response.status_code == 200:
        block_locations = response.json()['BlockLocations']['BlockLocation']
        block_distribution = defaultdict(int)
        for block in block_locations:
            for host in block['hosts']:
                block_distribution[host] += 1
        return dict(block_distribution)

status_params = {"op": "GETFILESTATUS"}
response = requests.get("http://boss:9870/webhdfs/v1/double.parquet", params=status_params)
file_status = response.json()['FileStatus']
total_length = file_status['length']
block_distribution = get_block_locations("/double.parquet", total_length)
block_distribution

{'2a68af0b41a3': 16, '9322cd244a1a': 16}

In [8]:
#q7
import time

import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq

hdfs = pa.fs.HadoopFileSystem(host="boss", port=9000)
start_time = time.time()
with hdfs.open_input_file("/double.parquet") as parquet_file:
    table = pq.read_table(parquet_file)
avg_loan_amount = pc.mean(table["loan_amount"]).as_py()
end_time = time.time()
q7_duration = end_time - start_time
avg_loan_amount

2024-10-19 00:35:07,716 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


204961.21752386744

In [9]:
#q8
hdfs = pa.fs.HadoopFileSystem(host="boss", port=9000)
start_time = time.time()
with hdfs.open_input_file("/double.parquet") as parquet_file:
    table = pq.read_table(parquet_file, columns=["loan_amount"])
avg_loan_amount = pc.mean(table["loan_amount"]).as_py()
end_time = time.time()
q8_duration = end_time - start_time
performance_improvement = q7_duration / q8_duration
performance_improvement

18.575664991457682